# 5. Blind rotation
A blind rotation procdeure is the core of FHEW bootstrapping and other many application of HE supporting non-arithmetic (not $\times$ or +) operations.

It is defined as follows.

**Definition (blind rotation)**
- Input: LWE ciphertext $(\beta, \vec{\alpha})$, blind rotation keys $brk$ under secret $\boldsymbol{z}$, and public polynomial $f$, where $\beta + \left< \vec{\alpha}, \vec{s}\right> = u$
- Output: $RLWE_{\boldsymbol{z}}(f\cdot X^u)$

It is compose of following three steps.
1. Make a encryption of $f\cdot X^{\beta}$, $ACC$. It can easily be done $ACC = (f\cdot X^{\beta}, 0)$.
2. Accumulation: homomophically multiply $X^{\alpha_i s_i}$ to $ACC$ and update it.
3. After accumulation, we get $RLWE_{\boldsymbol{z}}(f\cdot X^{\beta + \left< \vec{\alpha}, \vec{s}\right>}) = RLWE_{\boldsymbol{z}}(f\cdot X^u)$

In [1]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ

def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def normalize(v, logQ):
    # same as follows but no branch
    """
    if v > Q//2:
        v -= Q
    """
    # vmod Q when Q is a power-of-two
    Q = (1 << logQ)
    v &= Q-1
    # get msb
    msb = (v & Q//2) >> (logQ - 1)
    v -= (Q) * msb
    return v

def decrypt_from_fft(ctfft, sfft):
    assert len(ctfft.size()) == 2
    return normalize(negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q), logQ)

First, we generate $f$ for NAND gate.
$[-q/8, 3q/8]$ is mapped to $q/8$, and $[3q/8, 7q/8]$ is mapped to $-q/8$

NOTE: $f$ depends on the binary gate we want to perform.


In [2]:
q = 2*N
f_nand = torch.zeros(size=[N], dtype=torch.int32)

f_nand -= (Q>>3)
f_nand[3*q//8:] += (Q>>2)